In [ ]:
# math
import numpy as np
import pandas as pd
import datetime

# other
import json

from experiments_2022 import DATASETS_PATH, IMAGE_PATH
from experiments_2022.zone_level_analysis import (
    base,
    cleaning,
    viz,
    regression_functions,
)
from experiments_2022.datasets import (
    load_weather,
    load_building,
    pull_from_dataset,
)

from utils import (
    PROJECTS_2022,
    PROJECTS_VAV,
    NO_WEEKENDS,
    CONTROL_FOR_WEEKENDS,
    SUMMER_START_2022,
    SUMMER_END_2022,
    SINGLE_PLOT_TXT_SIZE,
    SINGLE_PLOT_LEGEND_SIZE,
    SINGLE_PLOT_ANNOTATION_SIZE,
    MULTI_PLOT_TXT_SIZE,
    MULTI_PLOT_LEGEND_SIZE,
    run_building_regressions,
    add_vertical_boxes,
    gini,
)

In [ ]:
pd.set_option("display.max_rows", 1000000)

In [ ]:
with open("./dicts_dfs/all_zones.json", "r") as f:
    ALL_ZONES = json.load(f)

with open("./dicts_dfs/zones_by_type.json", "r") as f:
    ZONES_BY_TYPE = json.load(f)

with open("./dicts_dfs/ahus.json", "r") as f:
    AHUS = json.load(f)

EXCLUDED_ZONES = {}
for project in PROJECTS_2022:
    ezs = pd.read_csv(f"./dicts_dfs/excluded/{project}.csv").set_index("Unnamed: 0")
    ezs.index.name = None
    EXCLUDED_ZONES[project] = ezs

## Testbed table

In [ ]:
testbed = pd.DataFrame(
    index=[
        "Lab Building",
        "1000m2",
        "Year of Construction",
        "Year of Last Retrofit",
        "# of AHs",
        "# of VAVs + FCs",
        "# of VAVs",
        "# of FCs",
        "# of Excluded Zones",
        "% of Excluded Zones",
        "Summer Daily Energy Demand (kWh/m2/day)",
        "% Cooling Demand",
        "% Electric Demand",
        "% Heating Demand",
        "Total Experiment Weekdays",
        "# of 74F Weekdays",
        "# of 76F Weekdays",
        "# of 78F Weekdays",
    ],
    columns=PROJECTS_2022,
)
testbed["TOTAL"] = np.nan

### Lab building

In [ ]:
testbed.loc["Lab Building", PROJECTS_2022] = "N"
testbed.loc["Lab Building", "LAB-1"] = "Y"
testbed.loc["Lab Building", "LAB-2"] = "Y"
testbed.loc["Lab Building", "LAB-3"] = "Y"

### m2

In [ ]:
testbed.loc["1000m2", "OFF-1"] = 144982 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "OFF-2"] = 135556 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "OFF-3"] = 28420 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "OFF-4"] = 169619 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "OFF-5"] = 84501 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "OFF-6"] = 105465 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "OFF-7"] = 63439 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "LAB-1"] = 75081 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "LAB-2"] = 63670 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "LAB-3"] = 76462 * base.M2_PER_SF / 1000
testbed.loc["1000m2", "TOTAL"] = testbed.loc["1000m2", PROJECTS_2022].sum()

### Year of construction

In [ ]:
testbed.loc["Year of Construction", "OFF-1"] = 2000
testbed.loc["Year of Construction", "OFF-2"] = 1996
testbed.loc["Year of Construction", "OFF-3"] = 1893
testbed.loc["Year of Construction", "OFF-4"] = 1996
testbed.loc["Year of Construction", "OFF-5"] = 2015
testbed.loc["Year of Construction", "OFF-6"] = 1998
testbed.loc["Year of Construction", "OFF-7"] = 1900
testbed.loc["Year of Construction", "LAB-1"] = 1965
testbed.loc["Year of Construction", "LAB-2"] = 1902
testbed.loc["Year of Construction", "LAB-3"] = 1963

### Year of last retrofit

In [ ]:
testbed.loc["Year of Last Retrofit", "OFF-1"] = 2014
testbed.loc["Year of Last Retrofit", "OFF-2"] = 2021
testbed.loc["Year of Last Retrofit", "OFF-3"] = 2015
testbed.loc["Year of Last Retrofit", "OFF-4"] = 2015
testbed.loc["Year of Last Retrofit", "OFF-5"] = np.nan
testbed.loc["Year of Last Retrofit", "OFF-6"] = 2012
testbed.loc["Year of Last Retrofit", "OFF-7"] = 2017
testbed.loc["Year of Last Retrofit", "LAB-1"] = 2020
testbed.loc["Year of Last Retrofit", "LAB-2"] = 2016
testbed.loc["Year of Last Retrofit", "LAB-3"] = 2018

### Number of equipment

In [ ]:
for project in PROJECTS_2022:
    # ahus
    testbed.loc["# of AHs", project] = len(AHUS[project])
    # vavs
    testbed.loc["# of VAVs", project] = len(ZONES_BY_TYPE[project]["VAVs"])
    # fcus
    testbed.loc["# of FCs", project] = len(ZONES_BY_TYPE[project]["FCUs"])
    # excluded
    testbed.loc["# of Excluded Zones", project] = len(
        (EXCLUDED_ZONES[project][EXCLUDED_ZONES[project] == 1]).dropna()
    )

for row in ["# of VAVs", "# of FCs", "# of AHs", "# of Excluded Zones"]:
    testbed.loc[row, "TOTAL"] = testbed.loc[row, PROJECTS_2022].sum()

testbed.loc["# of VAVs + FCs", :] = (
    testbed.loc["# of VAVs", :] + testbed.loc["# of FCs", :]
)
testbed.loc["% of Excluded Zones", :] = (
    (testbed.loc["# of Excluded Zones", :] / testbed.loc["# of VAVs + FCs", :])
    .astype(float)
    .round(2)
)

### Cooling

In [ ]:
cooling = cleaning.clean_df(
    df=load_building("2023", "C"),
    this_var="building-cooling",
    start_date=pd.Timestamp("05-01-2023"),
    end_date=pd.Timestamp("10-01-2023"),
    only_business_hours=False,
    no_weekends=True,
)
cooling = cooling[PROJECTS_2022]

In [ ]:
fig = viz.make_time_series(cooling, y_axis_title="Cooling (tons)")

In [ ]:
# fig

In [ ]:
cooling = cooling * base.MW_PER_TON * (10**3)
cooling["TOTAL"] = cooling.sum(axis=1, skipna=True)
cooling = cooling.groupby(cooling.index.date).sum()
for project in list(cooling.columns):
    testbed.loc["Average Cooling Demand (kWh/m2/day)", project] = cooling[
        project
    ].mean() / (1000 * testbed.loc["1000m2", project])

### Heating

In [ ]:
heating = cleaning.clean_df(
    df=load_building("2023", "H"),
    this_var="building-heating",
    start_date=pd.Timestamp("05-01-2023"),
    end_date=pd.Timestamp("10-01-2023"),
    only_business_hours=False,
    no_weekends=True,
)
heating = heating[PROJECTS_2022]

In [ ]:
fig = viz.make_time_series(heating, y_axis_title="Heating (kBtu)")

In [ ]:
# fig

In [ ]:
heating = heating * base.WH_PER_BTU
heating["TOTAL"] = heating.sum(axis=1, skipna=True)
heating = heating.groupby(heating.index.date).sum()
heating[heating == 0] = np.nan  # edge case
for project in list(heating.columns):
    testbed.loc["Average Heating Demand (kWh/m2/day)", project] = heating[
        project
    ].mean() / (1000 * testbed.loc["1000m2", project])

### Electricity

In [ ]:
electricity = cleaning.clean_df(
    df=load_building("2023", "E"),
    this_var="building-electricity",
    start_date=pd.Timestamp("05-01-2023"),
    end_date=pd.Timestamp("10-01-2023"),
    only_business_hours=False,
    no_weekends=True,
)
electricity = electricity[PROJECTS_2022]
electricity[electricity > 1000] = np.nan
electricity["OFF-1"][electricity["OFF-1"] > 200] = np.nan
electricity["OFF-2"][electricity["OFF-2"] > 400] = np.nan
electricity["OFF-3"][electricity["OFF-3"] > 20] = np.nan
electricity["OFF-4"][electricity["OFF-4"] > 250] = np.nan
electricity["OFF-5"][electricity["OFF-5"] > 250] = np.nan
electricity["OFF-6"][electricity["OFF-6"] > 300] = np.nan
electricity["OFF-7"][electricity["OFF-7"] > 200] = np.nan
electricity["LAB-1"][electricity["LAB-1"] > 500] = np.nan
electricity["LAB-2"][electricity["LAB-2"] > 250] = np.nan
electricity["LAB-3"][electricity["LAB-3"] > 300] = np.nan

In [ ]:
fig = viz.make_time_series(
    electricity, y_axis_title="Electricity (kW)", force_same_yaxes=False
)

In [ ]:
# fig

In [ ]:
electricity["TOTAL"] = electricity.sum(axis=1, skipna=True)
electricity = electricity.groupby(electricity.index.date).sum()
for project in list(electricity.columns):
    testbed.loc["Average Electric Demand (kWh/m2/day)", project] = electricity[
        project
    ].mean() / (1000 * testbed.loc["1000m2", project])

### Total 

In [ ]:
testbed.loc["Summer Daily Energy Demand (kWh/m2/day)", PROJECTS_2022] = (
    testbed.loc["Average Cooling Demand (kWh/m2/day)", PROJECTS_2022]
    + testbed.loc["Average Heating Demand (kWh/m2/day)", PROJECTS_2022]
    + testbed.loc["Average Electric Demand (kWh/m2/day)", PROJECTS_2022]
)
testbed.loc["Summer Daily Energy Demand (kWh/m2/day)", "TOTAL"] = (
    (
        testbed.loc["Summer Daily Energy Demand (kWh/m2/day)", PROJECTS_2022]
        * testbed.loc["1000m2", PROJECTS_2022]
        * 1000
    ).sum()
) / (testbed.loc["1000m2", PROJECTS_2022].sum() * 1000)

for utility in ["Cooling", "Heating", "Electric"]:
    testbed.loc[f"% {utility} Demand", :] = (
        (
            testbed.loc[f"Average {utility} Demand (kWh/m2/day)", :]
            / testbed.loc["Summer Daily Energy Demand (kWh/m2/day)", :]
        )
        .astype(float)
        .round(3)
    )
    testbed.drop(index=f"Average {utility} Demand (kWh/m2/day)", inplace=True)

### # of Day Types

In [ ]:
schedule = pd.read_csv(
    DATASETS_PATH / "csvs/2022_experiment_csvs/sp_schedule_daily.csv"
).set_index("Unnamed: 0")
schedule.index = pd.to_datetime(schedule.index)
for project in PROJECTS_2022:
    ser = schedule[project]
    for sp in [74, 76, 78]:
        this_ser = ser[ser == sp]
        this_ser = this_ser[this_ser.index.dayofweek < 5]
        testbed.loc[f"# of {sp}F Weekdays", project] = len(this_ser)
        testbed.loc[f"# of {sp}F Weekdays", "TOTAL"] = testbed.loc[
            f"# of {sp}F Weekdays", PROJECTS_2022
        ].sum()

testbed.loc["Total Experiment Weekdays", :] = testbed.loc[
    ["# of 74F Weekdays", "# of 76F Weekdays", "# of 78F Weekdays"], :
].sum()

In [ ]:
testbed

## Min airflow

In [ ]:
min_airflow = pd.read_csv(DATASETS_PATH / "csvs/min_airflow_2023.csv").drop(
    columns=["Unnamed: 0"]
)

In [ ]:
min_airflow_dict = {}
for project in PROJECTS_VAV:
    this_min_airflow = (
        min_airflow[min_airflow["Building"] == project]
        .set_index("VAV")["Minimum Airflow (cfm)"]
        .to_frame()
    )
    this_min_airflow[this_min_airflow < cleaning.LOWER_LIMIT["zone-airflow"]] = np.nan
    this_min_airflow[this_min_airflow > cleaning.UPPER_LIMIT["zone-airflow"]] = np.nan
    min_airflow_dict[project] = this_min_airflow

In [ ]:
fig = viz.make_dot_plot(
    y_data=min_airflow_dict,
    y_axis_title="Min Airflow (cfm)",
    vertical_spacing=0.1,
    force_same_yaxes=True,
)

In [ ]:
# fig

In [ ]:
min_airflow_summary = pd.Series(index=PROJECTS_VAV)
for project in PROJECTS_VAV:
    min_airflow_summary[project] = min_airflow_dict[project].sum()[0] / (
        testbed.loc["1000m2", project] * 1000 / base.M2_PER_SF
    )

In [ ]:
min_airflow_summary.to_frame()

## Building wide cooling

In [ ]:
cooling_df = cleaning.clean_by_column(
    df=load_building("2022", "C")[PROJECTS_2022],
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
)

cooling_df = cleaning.clean_df(
    df=cooling_df,
    this_var="building-cooling",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=False,
    no_weekends=False,
)

In [ ]:
fig = viz.make_time_series(cooling_df, y_axis_title="Cooling [tons]")

In [ ]:
# fig

In [ ]:
cooling_df = cooling_df * base.MW_PER_TON * 1000  # kWh
cooling = {}
for project in PROJECTS_2022:
    cooling[project] = cooling_df[project].to_frame() / testbed.loc["1000m2", project]
for project in PROJECTS_2022:
    for day in ["06-23-2022"]:
        cooling[project].loc[day, :] = np.nan

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=False,
    SI_units=True,
)["temperature"]

In [ ]:
(
    percent_deltas_cooling,
    percent_deltas_high_cooling,
    percent_deltas_low_cooling,
    percent_summary,
    fig,
) = run_building_regressions(
    cooling,
    T,
    mode="Percent Change",
    summary_statistic="Mean",
    y_axis_title="Daytime Average<br>Cooling Demand<br>[kW/1000m2]",
    use_raw=False,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
        font_size=MULTI_PLOT_LEGEND_SIZE,
    ),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureG1.png")

In [ ]:
percent_summary.columns = percent_summary.columns.str.upper()

In [ ]:
percent_summary

In [ ]:
(
    absolute_deltas_cooling,
    absolute_deltas_high_cooling,
    absolute_deltas_low_cooling,
    summary,
    fig,
) = run_building_regressions(
    cooling,
    T,
    mode="Absolute Change",
    summary_statistic="Mean",
    y_axis_title="Daytime Average<br>Cooling Demand<br>[kW/1000m2]",
    use_raw=False,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
        font_size=viz.LEGEND_SZ + 2,
    ),
)

In [ ]:
# fig

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=percent_deltas_cooling["78"].to_frame(),
    y_error_up_data=percent_deltas_high_cooling["78"].to_frame(),
    y_error_down_data=percent_deltas_low_cooling["78"].to_frame(),
    marker_legend={"color": {"78": "Black"}},
    y_axis_title="Percent Change in<br>Cooling Demand [%]",
    point_start=(1 / 2),
    offset_delta=(1 / 2),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_2022)],
    y_range=[-70, 20],
    marker_size=10,
    error_thickness=2.5,
    whisker_len=8,
    dont_add_to_legend=["78"],
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_2022) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
percent_deltas_cooling["78"]

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure4A.png")

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=percent_deltas_cooling["78"].to_frame(),
    y_error_up_data=percent_deltas_high_cooling["78"].to_frame(),
    y_error_down_data=percent_deltas_low_cooling["78"].to_frame(),
    marker_legend={"color": {"78": "Black"}},
    y_axis_title="Percent Change in<br>Cooling Demand [%]",
    point_start=(1 / 2),
    offset_delta=(1 / 2),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_2022)],
    y_range=[-70, 20],
    marker_size=14,
    error_thickness=4,
    whisker_len=8,
    dont_add_to_legend=["78"],
    text_size=MULTI_PLOT_TXT_SIZE + 6,
)
fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_2022) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
fig.write_image(f"{IMAGE_PATH}/GraphAbstractTopLeft.png")

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=absolute_deltas_cooling["78"].to_frame(),
    y_error_up_data=absolute_deltas_high_cooling["78"].to_frame(),
    y_error_down_data=absolute_deltas_low_cooling["78"].to_frame(),
    marker_legend={"color": {"78": "Black"}},
    y_axis_title="Absolute Change in<br>Cooling Demand [kW/1000m2]",
    point_start=(1 / 2),
    offset_delta=(1 / 2),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_2022)],
    y_range=[-14, 2],
    error_thickness=2.5,
    whisker_len=8,
    dont_add_to_legend=["78"],
    text_size=SINGLE_PLOT_TXT_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
)
fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_2022) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
# absolute_deltas_cooling["78"]

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureA1.png")

### Check regression

In [ ]:
project = "OFF-2"

In [ ]:
df = cooling[project]

In [ ]:
binary_df = regression_functions.get_2021_2022_binary_df(
    project=project,
    experiment_year="2022",
    freq="daily",
    baseline_column="CSP = 74F",
    drop_baseline_column=True,
    no_weekends=NO_WEEKENDS[project],
    control_for_weekends=CONTROL_FOR_WEEKENDS[project],
)

reg_results = regression_functions.general_Delta_fn(
    df=df,
    T=T,
    binary=binary_df,
    mode="Absolute Change",
    summary_statistic="Mean",
)

In [ ]:
fig = viz.plot_experiment_regression(
    experiment_results=reg_results,
    df=df,
    T=T,
    binary=binary_df,
    line_legend={
        "name": {
            "Control": "CSP = 23.3°C",
            "CSP = 76F": "CSP = 24.4°C",
            "CSP = 78F": "CSP = 25.5°C",
        },
        "color": {
            "Control": "RoyalBlue",
            "CSP = 76F": "DarkOrange",
            "CSP = 78F": "Firebrick",
        },
    },
    mode="Absolute Change",
    summary_statistic="Mean",
    line_width=2.5,
    y_axis_title="Daytime Average<br>Cooling Demand (kW)",
    x_axis_title="Daytime Average OAT [°C]",
    height=550,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.225,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

new_titles = ["Building Level"]
for i, annotation in enumerate(fig.layout.annotations):
    annotation.text = new_titles[i]

In [ ]:
# fig

## MAT - SAT

In [ ]:
dat = pull_from_dataset("2022", PROJECTS_VAV, "ahu-dat")
dat = cleaning.clean_dfs(
    dfs=dat,
    this_var="ahu-dat",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=True,
)
mat = pull_from_dataset("2022", PROJECTS_VAV, "ahu-mat")
mat = cleaning.clean_dfs(
    dfs=mat,
    this_var="ahu-mat",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=True,
)
mat_dat = {}
for project in PROJECTS_VAV:
    mat_dat[project] = mat[project] - dat[project]

airflow = pull_from_dataset("2022", PROJECTS_VAV, "ahu-airflow")
airflow = cleaning.clean_dfs(
    dfs=airflow,
    this_var="ahu-airflow",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=True,
)
mat = base.calculate_airflow_weighted_average(mat, airflow)
dat = base.calculate_airflow_weighted_average(dat, airflow)
mat_dat = base.calculate_airflow_weighted_average(mat_dat, airflow)

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=False,
    SI_units=True,
)["temperature"]

In [ ]:
(
    percent_deltas_mat_dat,
    percent_deltas_high_mat_dat,
    percent_deltas_low_mat_dat,
    percent_summary,
    fig,
) = run_building_regressions(
    mat_dat,
    T,
    mode="Percent Change",
    summary_statistic="Mean",
    y_axis_title="Average MAT - SAT [°C]",
    use_raw=False,
)

In [ ]:
percent_deltas_mat_dat

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=percent_deltas_mat_dat["78"].to_frame(),
    y_error_up_data=percent_deltas_high_mat_dat["78"].to_frame(),
    y_error_down_data=percent_deltas_high_mat_dat["78"].to_frame(),
    marker_legend={"color": {"78": "Black"}},
    y_axis_title="Percent Change in (MAT - SAT) [%]",
    point_start=(1 / 2),
    offset_delta=(1 / 2),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_VAV))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_VAV)],
    y_range=[-70, 20],
    error_thickness=2.5,
    whisker_len=8,
    dont_add_to_legend=["78"],
    text_size=SINGLE_PLOT_TXT_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
)
fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_VAV) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure6A.png")

In [ ]:
(
    absolute_deltas_mat_dat,
    absolute_deltas_high_mat_dat,
    absolute_deltas_low_mat_dat,
    absolute_summary_mat_dat,
    fig,
) = run_building_regressions(
    mat_dat,
    T,
    mode="Absolute Change",
    summary_statistic="Mean",
    y_axis_title="Average MAT - SAT [°C]",
    use_raw=False,
)

(
    absolute_deltas_dat,
    absolute_deltas_high_dat,
    absolute_deltas_low_dat,
    absolute_summary_dat,
    fig,
) = run_building_regressions(
    dat,
    T,
    mode="Absolute Change",
    summary_statistic="Mean",
    y_axis_title="Average SAT [°C]",
    use_raw=False,
)

(
    absolute_deltas_mat,
    absolute_deltas_high_mat,
    absolute_deltas_low_mat,
    absolute_summary_mat,
    fig,
) = run_building_regressions(
    mat,
    T,
    mode="Absolute Change",
    summary_statistic="Mean",
    y_axis_title="Average MAT [°C]",
    use_raw=False,
)

In [ ]:
deltas = pd.DataFrame(
    index=absolute_deltas_mat_dat.index,
    columns=["MAT - SAT", "SAT", "MAT"],
)
deltas["MAT - SAT"] = absolute_deltas_mat_dat["78"]
deltas["SAT"] = absolute_deltas_dat["78"]
deltas["MAT"] = absolute_deltas_mat["78"]

deltas_high = pd.DataFrame(
    index=absolute_deltas_mat_dat.index,
    columns=["MAT - SAT", "SAT", "MAT"],
)
deltas_high["MAT - SAT"] = absolute_deltas_high_mat_dat["78"]
deltas_high["SAT"] = absolute_deltas_high_dat["78"]
deltas_high["MAT"] = absolute_deltas_high_mat["78"]

deltas_low = pd.DataFrame(
    index=absolute_deltas_mat_dat.index,
    columns=["MAT - SAT", "SAT", "MAT"],
)
deltas_low["MAT - SAT"] = absolute_deltas_low_mat_dat["78"]
deltas_low["SAT"] = absolute_deltas_low_dat["78"]
deltas_low["MAT"] = absolute_deltas_low_mat["78"]

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=deltas,
    y_error_up_data=deltas_high,
    y_error_down_data=deltas_low,
    marker_legend={
        "color": {
            "MAT - SAT": "Black",
            "SAT": "Coral",
            "MAT": "Orchid",
        }
    },
    y_axis_title="Absolute Change in<br>Temperature [°C]",
    point_start=(1 / 4),
    offset_delta=(1 / 4),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_VAV))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_VAV)],
    y_range=[-6, 6],
    error_thickness=2.5,
    whisker_len=8,
    text_size=SINGLE_PLOT_TXT_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
)
fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_VAV) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
deltas

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureA4.png")

### Check regression


In [ ]:
project = "OFF-2"

In [ ]:
df = mat_dat[project]

In [ ]:
binary_df = regression_functions.get_2021_2022_binary_df(
    project=project,
    experiment_year="2022",
    freq="daily",
    baseline_column="CSP = 74F",
    drop_baseline_column=True,
    no_weekends=NO_WEEKENDS[project],
    control_for_weekends=CONTROL_FOR_WEEKENDS[project],
)

reg_results = regression_functions.general_Delta_fn(
    df=df,
    T=T,
    binary=binary_df,
    mode="Absolute Change",
    summary_statistic="Mean",
)

In [ ]:
fig = viz.plot_experiment_regression(
    experiment_results=reg_results,
    df=df,
    T=T,
    binary=binary_df,
    line_legend={
        "name": {
            "Control": "CSP = 23.3°C",
            "CSP = 76F": "CSP = 24.4°C",
            "CSP = 78F": "CSP = 25.5°C",
        },
        "color": {
            "Control": "RoyalBlue",
            "CSP = 76F": "DarkOrange",
            "CSP = 78F": "Firebrick",
        },
    },
    mode="Absolute Change",
    summary_statistic="Mean",
    line_width=2.5,
    y_axis_title="Daytime Average<br>(MAT - SAT) [°C]",
    x_axis_title="Daytime Average OAT [°C]",
    height=550,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.225,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

new_titles = ["AHU Level"]
for i, annotation in enumerate(fig.layout.annotations):
    annotation.text = new_titles[i]

In [ ]:
# fig

## Airflow 

In [ ]:
airflow = pull_from_dataset("2022", PROJECTS_VAV, "zone-airflowsp")
airflow = cleaning.clean_dfs(
    dfs=airflow,
    this_var="zone-airflowsp",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    SI_units=True,
)
for project in PROJECTS_VAV:
    airflow[project] = airflow[project].sum(axis=1).to_frame()
    airflow[project].columns = [project]

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=False,
    SI_units=True,
)["temperature"]

In [ ]:
(
    percent_deltas_airflow,
    percent_deltas_high_airflow,
    percent_deltas_low_airflow,
    percent_airflow,
    fig,
) = run_building_regressions(
    airflow,
    T,
    mode="Percent Change",
    summary_statistic="Mean",
    y_axis_title="Average Airflow (m3/hr)",
    use_raw=False,
)

In [ ]:
# fig

In [ ]:
percent_deltas_airflow["78"]

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=percent_deltas_airflow["78"].to_frame(),
    y_error_up_data=percent_deltas_high_airflow["78"].to_frame(),
    y_error_down_data=percent_deltas_low_airflow["78"].to_frame(),
    y_axis_title="Percent Change in Airflow [%]",
    marker_legend={"color": {"78": "Black"}},
    point_start=(1 / 2),
    offset_delta=(1 / 2),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_VAV))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_VAV)],
    y_range=[-70, 20],
    error_thickness=2.5,
    whisker_len=8,
    dont_add_to_legend=["78"],
    text_size=SINGLE_PLOT_TXT_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
)
fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_VAV) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure5A.png")

## Cooling, MAT - SAT, Airflow correlation

In [ ]:
corr = float(
    percent_deltas_cooling.loc[PROJECTS_VAV, "78"].corr(
        percent_deltas_mat_dat.loc[PROJECTS_VAV, "78"]
        + percent_deltas_airflow.loc[PROJECTS_VAV, "78"]
    )
)

In [ ]:
fig = viz.make_scatter_plot(
    y_data=-percent_deltas_cooling.loc[PROJECTS_VAV, "78"].to_frame(),
    x_data=-(
        percent_deltas_mat_dat.loc[PROJECTS_VAV, "78"]
        + percent_deltas_airflow.loc[PROJECTS_VAV, "78"]
    ).to_frame(),
    y_axis_title="Percent Reduction<br>in Cooling [%]",
    x_axis_title="Percent Reduction in (MAT - SAT) [%] +<br>Percent Reduction in Airflow [%]",
    y_range=[-10, 70],
    x_range=[0, 100],
    text_size=SINGLE_PLOT_TXT_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE
    # height=550,
    # width=850
)
fig = fig.add_annotation(
    text=f"Correlation Coefficient = {corr:.2f}",
    xref="x1",
    yref="y1",
    x=5,
    y=60,
    showarrow=False,
    xanchor="left",
    yanchor="top",
    font=dict(size=SINGLE_PLOT_ANNOTATION_SIZE, color="black"),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure11B.png")

## Cooling, gini correlation

In [ ]:
dfs = pull_from_dataset(
    "2022",
    PROJECTS_2022,
    "zone-simple_cooling_requests",
    clean_data=True,
)

zonal_schedules = {}
for project in PROJECTS_2022:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2022", freq="hourly", df=dfs[project]
    )

df_filter = cleaning.create_sp_filter(zonal_schedules, sps=[74], reverse_filter=False)

dfs_control = cleaning.clean_dfs(
    dfs=dfs,
    this_var="zone-dummy",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=True,
    df_filter=df_filter,
)

dfs_control_mean = base.run_passive_test_on_dfs(
    dfs=dfs_control, this_test="Mean", col_name="Mean"
)

In [ ]:
gini_ser = pd.Series(index=PROJECTS_2022)
for project in PROJECTS_2022:
    gini_ser[project] = gini(dfs_control_mean[project].iloc[:, 0].dropna())

In [ ]:
fig = viz.make_scatter_plot(
    y_data=-percent_deltas_cooling["78"].to_frame(),
    x_data=gini_ser.to_frame(),
    y_axis_title="Percent Reduction<br>in Cooling [%]",
    x_axis_title="Gini Coefficient [Unitless]<br>       ",
    y_range=[-10, 70],
    x_range=[0, 1],
    text_size=SINGLE_PLOT_TXT_SIZE,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
)

fig = fig.add_annotation(
    text=f"Correlation Coefficient = {gini_ser.corr(-percent_deltas_cooling['78']):.2f}",
    xref="x1",
    yref="y1",
    x=0.05,
    y=60,
    showarrow=False,
    xanchor="left",
    yanchor="top",
    font=dict(size=SINGLE_PLOT_ANNOTATION_SIZE, color="black"),
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/Figure11A.png")